<a href="https://colab.research.google.com/github/Sasidhar0709/Project-Stock-market-Analysis/blob/main/Project_Final_21063192.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import yfinance as yf
from scipy.interpolate import interp1d
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

In [4]:

ftse100_ticker_symbols = [
    'BARC.L', 'LLOY.L','LSEG.L', 'MNG.L', 'MRO.L', 'MNDI.L', 'MCRO.L', 'NG.L', 'NXT.L', 'NPN.L', 'OCDO.L', 'PHNX.L']


end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(days=10*365)


stocks_data = {}

for ticker in ftse100_ticker_symbols:
    print(f"Fetching data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    stocks_data[ticker] = data


for ticker, data in stocks_data.items():
    data.to_csv(f'{ticker}_stock_data.csv')
    print(f"Data for {ticker} saved to {ticker}_stock_data.csv")


[*********************100%%**********************]  1 of 1 completed

Fetching data for BARC.L...
Fetching data for LLOY.L...



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching data for LSEG.L...
Fetching data for MNG.L...
Fetching data for MRO.L...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

Fetching data for NG.L...
Fetching data for NXT.L...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NPN.L']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2014-06-28 22:27:05.850718 -> 2024-06-25 22:27:05.850718)')
[*********************100%%**********************]  1 of 1 completed


Fetching data for NPN.L...
$NPN.L: possibly delisted; No price data found  (1d 2014-06-28 22:27:05.850718 -> 2024-06-25 22:27:05.850718)
Fetching data for OCDO.L...
Fetching data for PHNX.L...


[*********************100%%**********************]  1 of 1 completed


Data for BARC.L saved to BARC.L_stock_data.csv
Data for LLOY.L saved to LLOY.L_stock_data.csv
Data for LSEG.L saved to LSEG.L_stock_data.csv
Data for MNG.L saved to MNG.L_stock_data.csv
Data for MRO.L saved to MRO.L_stock_data.csv
Data for MNDI.L saved to MNDI.L_stock_data.csv
Data for MCRO.L saved to MCRO.L_stock_data.csv
Data for NG.L saved to NG.L_stock_data.csv
Data for NXT.L saved to NXT.L_stock_data.csv
Data for NPN.L saved to NPN.L_stock_data.csv
Data for OCDO.L saved to OCDO.L_stock_data.csv
Data for PHNX.L saved to PHNX.L_stock_data.csv
